In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11277,2024-02-27,Eua Nba,00:00,Sacramento Kings,Miami Heat,1.34,3.43,223.5,1.85,2.09,-7.5,2.01,7.00,WjP1PTPj,0.746269,0.291545,0.540541,0.478469,0.037814,0.8,0.2,NaN,NaN,253.000,64.344706,0.254327,286.560,118.101257,0.412134,169.092,283.454,152.40,234.26,0.0,0.0,0.0,0.0,0.619645,0.086145,0.783233,-0.69,-0.138,-2.463768,0.722553,0.5,-0.222553,4.69,0.938,2.590618,0.421351,0.7,0.278649
11278,2024-02-27,Eua Nba,21:00,Cleveland Cavaliers,Dallas Mavericks,1.54,2.26,232.5,1.80,2.01,-3.5,1.98,2.76,pfDqlD2i,0.649351,0.442478,0.555556,0.497512,0.091829,0.6,0.2,NaN,NaN,144.646,9.475301,0.065507,188.728,63.729947,0.337681,215.062,225.752,133.92,197.58,0.0,0.0,0.0,0.0,0.267956,0.077949,0.232719,2.33,0.466,1.158798,0.681083,0.9,0.218917,0.27,0.054,23.333333,0.460988,0.5,0.039012
11279,2024-02-27,Eua Nba,21:30,Atlanta Hawks,Utah Jazz,1.66,2.04,237.5,1.82,1.99,-4.5,2.04,2.79,KC5doBnA,0.602410,0.490196,0.549451,0.502513,0.092606,0.6,0.6,NaN,NaN,182.288,29.541651,0.162060,261.664,58.812542,0.224764,232.542,233.202,199.47,356.50,0.0,0.0,0.0,0.0,0.145244,0.063101,0.219598,0.08,0.016,41.250000,0.557657,0.6,0.042343,-2.34,-0.468,-2.222222,0.403401,0.3,-0.103401
11280,2024-02-27,Eua Ncaa,21:00,Bowling Green,Miami (Ohio),1.63,2.38,144.5,1.91,1.91,-4.5,2.00,3.30,vJaiTJ1a,0.613497,0.420168,0.523560,0.523560,0.033665,1.0,0.0,NaN,NaN,116.180,22.874921,0.196892,155.218,78.926636,0.508489,125.010,185.518,95.70,103.24,0.0,0.0,0.0,0.0,0.264504,0.000000,0.346883,-0.55,-0.110,-5.727273,0.689696,0.8,0.110304,-1.30,-0.260,-5.307692,0.333264,0.3,-0.033264
11281,2024-02-27,Eua Ncaa,21:00,Central Michigan,Ball State,1.63,2.38,134.5,1.91,1.91,-4.5,2.00,3.30,I9beSwH5,0.613497,0.420168,0.523560,0.523560,0.033665,0.2,0.2,NaN,NaN,208.066,135.513396,0.651300,122.380,33.528525,0.273971,130.214,138.938,106.02,134.40,0.0,0.0,0.0,0.0,0.264504,0.000000,0.346883,4.38,0.876,0.719178,0.571456,0.8,0.228544,0.69,0.138,10.000000,0.369875,0.4,0.030125
11282,2024-02-27,Eua Ncaa,21:00,Eastern Michigan,Western Michigan,2.05,1.80,143.5,1.91,1.91,1.5,1.91,1.63,nkjCOeGU,0.487805,0.555556,0.523560,0.523560,0.043360,0.2,0.2,NaN,NaN,200.836,66.803941,0.332629,179.778,72.792769,0.404904,125.272,287.368,123.00,125.58,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,-1.77,-0.354,-2.966102,0.000000,0.0,0.000000,-5.00,-1.000,-0.800000,0.257513,0.2,-0.057513
11283,2024-02-27,Eua Ncaa,21:00,Fordham,George Mason,2.05,1.80,137.5,1.91,1.91,1.5,1.91,1.63,vL0JDx3S,0.487805,0.555556,0.523560,0.523560,0.043360,0.8,0.4,NaN,NaN,148.070,16.874321,0.113962,143.082,33.708613,0.235589,128.024,142.754,153.87,145.73,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,-3.63,-0.726,-1.446281,0.685227,0.3,-0.385227,-3.09,-0.618,-1.294498,0.000000,0.0,0.000000
11284,2024-02-27,Eua Ncaa,21:00,Indiana,Wisconsin,2.47,1.58,140.5,1.80,1.95,3.5,1.95,1.32,vHWCDExE,0.404858,0.632911,0.555556,0.512821,0.037770,0.6,0.6,NaN,NaN,235.840,161.003023,0.682679,99.380,22.585048,0.227259,162.708,105.460,131.76,86.80,0.0,0.0,0.0,0.0,0.310778,0.056569,0.272463,-1.49,-0.298,-4.932886,0.610237,0.6,-0.010237,-3.49,-0.698,-0.830946,0.000000,0.0,0.000000
11285,2024-02-27,Eua Ncaa,21:00,LSU,Georgia,1.63,2.38,154.5,1.91,1.91,-4.5,2.00,3.30,CbtfNJQi,0.613497,0.420168,0.523560,0.523560,0.033665,0.8,0.6,NaN,NaN,305.282,161.393238,0.528669,210.188,150.025122,0.713766,158.684,265.234,264.04,166.50,0.0,0.0,0.0,0.0,0.264504,0.000000,0.346883,-0.62,-0.124,-5.080645,0.727016,0.7,-0.027016,1.40,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Sacramento Kings,Miami Heat,223.5,1.85,1.0,Over
1,21:00,Eua Nba,Cleveland Cavaliers,Dallas Mavericks,232.5,1.80,1.0,Over
2,21:30,Eua Nba,Atlanta Hawks,Utah Jazz,237.5,1.82,1.0,Over
3,21:00,Eua Ncaa,Bowling Green,Miami (Ohio),144.5,1.91,1.0,Over
4,21:00,Eua Ncaa,Central Michigan,Ball State,134.5,1.91,1.0,Over
5,21:00,Eua Ncaa,LSU,Georgia,154.5,1.91,1.0,Over
6,21:00,Eua Ncaa,Mississippi St.,Kentucky,156.5,1.80,1.0,Over
7,21:00,Eua Ncaa,Ohio,Akron,143.5,1.91,1.0,Over
8,21:00,Eua Ncaa,Syracuse,Virginia Tech,153.5,1.80,1.0,Over
9,22:30,Eua Ncaa,Eastern Illinois,Siu Edwardsville,140.5,1.91,1.0,Over
